# ***Rigorous Data Cleaning***

### Process Highlights:
- Customer Data: direct standardization of dates, flagged NULL columns as missing in a separate column before replacing with "unknown"
- Usage Logs: flagged missing European September logs before forward filling with August observations, created flags for daily login data
- Support Tickets: categorized tickets and flagged by issues

### Imports

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

### Load Data

In [2]:
df_customers = pd.read_csv('../data/dataset1.csv')
df_usage_q1q2 = pd.read_csv('../data/dataset2a for q1 q2.csv')
df_usage_q3q4 = pd.read_csv('../data/dataset2b for q3 q4.csv')
df_tickets = pd.read_csv('../data/dataset3.csv')

### Customer Data Cleaning

In [3]:
# Fix corrupted company size buckets
size_map = {
    '10-Jan': '1-10',
    'Nov-50': '11-50'
}
df_customers['company_size_bucket'] = df_customers['company_size_bucket'].replace(size_map)

# Flag missing industry entries prior to forward filling
df_customers['industry_missing'] = df_customers['industry'].isna()

# Fill missing industry with 'Unknown'
df_customers['industry'] = df_customers['industry'].fillna('Unknown')

# Convert dates
df_customers['contract_start_date'] = pd.to_datetime(df_customers['contract_start_date'])
df_customers['contract_end_date'] = pd.to_datetime(df_customers['contract_end_date'])

# Create time-based features
df_customers['contract_duration_days'] = (
    df_customers['contract_end_date'] - df_customers['contract_start_date']
).dt.days
df_customers['acquisition_quarter'] = df_customers['contract_start_date'].dt.to_period('Q')
df_customers['acquisition_month'] = df_customers['contract_start_date'].dt.to_period('M')

# Check for invalid dates
invalid_dates = df_customers[df_customers['contract_end_date'] < df_customers['contract_start_date']]
if len(invalid_dates) > 0:
    print(f"WARNING: {len(invalid_dates)} records have end_date before start_date")

### Usage Logs Cleaning

In [4]:
# Merge datasets
df_usage = pd.concat([df_usage_q1q2, df_usage_q3q4], ignore_index=True)

# Convert dates to datetime
df_usage['date'] = pd.to_datetime(df_usage['date'])

# Handle corrupted EU data by sorting by customer and date 
df_usage = df_usage.sort_values(['customer_id', 'date']).reset_index(drop=True)

# Merge with customer data to get region information for EU corruption flagging
df_usage = df_usage.merge(
    df_customers[['customer_id', 'region', 'is_eu']], 
    on='customer_id', 
    how='left'
)

# Check for any usage records without matching customer
missing_region = df_usage['region'].isna().sum()
if missing_region > 0:
    print(f"WARNING: {missing_region} usage records have no matching customer")

# Flag corrupted EU September data
df_usage['is_eu_sept_corrupted'] = (
    (df_usage['is_eu'] == 1) & 
    (df_usage['date'].dt.month == 9) &
    (df_usage['logins'].isna() | df_usage['feature_events'].isna() | df_usage['session_minutes'].isna())
)
corrupted_count = df_usage['is_eu_sept_corrupted'].sum()
total_eu_sept = ((df_usage['is_eu'] == 1) & (df_usage['date'].dt.month == 9)).sum()

# Forward fill corrupted records
df_usage['logins'] = df_usage.groupby('customer_id')['logins'].ffill()
df_usage['feature_events'] = df_usage.groupby('customer_id')['feature_events'].ffill()
df_usage['session_minutes'] = df_usage.groupby('customer_id')['session_minutes'].ffill()

# Create separate metrics without corrupted data
df_usage_clean = df_usage[~df_usage['is_eu_sept_corrupted']].copy()

usage_summary = df_usage.groupby('customer_id').agg({
    'date': 'count', 
    'logins': ['sum', 'mean', 'std'],
    'feature_events': ['sum', 'mean'],
    'session_minutes': ['sum', 'mean'],
    'is_eu_sept_corrupted': 'sum' 
}).reset_index()

# Flatten column names
usage_summary.columns = [
    'customer_id', 
    'active_days', 
    'total_logins', 'avg_logins_per_day', 'std_logins',
    'total_feature_events', 'avg_feature_events',
    'total_session_minutes', 'avg_session_minutes',
    'corrupted_records_count'
]

# Create clean metrics without corrupted data
usage_summary_clean = df_usage_clean.groupby('customer_id').agg({
    'date': 'count',
    'logins': ['sum', 'mean'],
    'session_minutes': 'sum'
}).reset_index()

usage_summary_clean.columns = [
    'customer_id',
    'active_days_clean',
    'total_logins_clean', 'avg_logins_per_day_clean',
    'total_session_minutes_clean'
]

# Merge clean metrics into main summary
usage_summary = usage_summary.merge(usage_summary_clean, on='customer_id', how='left')

# Flag customers with no logins or low engagement
usage_summary['never_logged_in'] = usage_summary['total_logins'] == 0
usage_summary['low_engagement'] = usage_summary['avg_logins_per_day'] < 1

# Detect outliers based on 99th percentile
outlier_threshold = usage_summary['total_logins'].quantile(0.99)
usage_summary['potential_outlier'] = usage_summary['total_logins'] > outlier_threshold


### Support Tickets Cleaning

In [5]:
# Convert ticket creation column to datetime
df_tickets['created_at'] = pd.to_datetime(df_tickets['created_at'])

# Extract month and quarter from created_at for time-series analysis
df_tickets['ticket_month'] = df_tickets['created_at'].dt.to_period('M')
df_tickets['ticket_quarter'] = df_tickets['created_at'].dt.to_period('Q')

# Create flags for key issue types (based on ticket_text in dataset3)
# Dashboard Issues
dashboard_texts = {
    "Dashboard loads very slowly during peak hours.",
    "The new dashboard is confusing for my staff."
}
df_tickets['is_dashboard_issue'] = df_tickets['ticket_text'].isin(dashboard_texts)

# Product Issues
product_texts = {
    "Inventory sync takes too long to complete.",
    "The reporting page keeps timing out.",
    "Mobile app crashes when I try to view inventory levels.",
    "It's hard to find where to update product SKUs.",
    "Bulk upload flow is not intuitive.",
    "Search results are not clearly sorted."
}
df_tickets['is_product_bug'] = df_tickets['ticket_text'].isin(product_texts)

# Onboarding Issues
onboarding_texts = {
    "Sales promised a custom report that doesn't exist.",
    "We expected multi-warehouse support out of the box.",
    "Pricing discussed in the demo does not match our invoice.",
    "We were told onboarding would be fully managed, but it wasn't.",
    "VAT appears to be calculated incorrectly.",
    "We were charged for extra users we don't have.",
    "Our discount was not applied to this renewal.",
    "I need to change our billing email and can't find the option."
}
df_tickets['is_onboarding_issue'] = df_tickets['ticket_text'].isin(onboarding_texts)

# Calculate tickets per customer
ticket_summary = df_tickets.groupby('customer_id').agg({
    'ticket_id': 'count',
    'is_dashboard_issue': 'sum',
    'is_product_bug': 'sum',
    'is_onboarding_issue': 'sum',
    'sentiment': 'mean',
    'first_response_hours': 'mean',
    'resolution_hours': 'mean',
    'resolved': 'mean'  
}).reset_index()

ticket_summary.columns = [
    'customer_id', 
    'total_tickets',
    'dashboard_issue_count',
    'product_bug_count',
    'onboarding_issue_count',
    'avg_sentiment',
    'avg_first_response_hours',
    'avg_resolution_hours',
    'resolution_rate'
]

# Flag high-ticket customers
ticket_summary['high_ticket_volume'] = ticket_summary['total_tickets'] > ticket_summary['total_tickets'].quantile(0.75)

# Determine primary issue type
def get_primary_complaint(row):
    if row['dashboard_issue_count'] > 0:
        return 'Dashboard_Performance'
    elif row['product_bug_count'] > 0:
        return 'Product_Bug'
    elif row['onboarding_issue_count'] > 0:
        return 'Onboarding'
    else:
        return 'Other'

ticket_summary['primary_complaint_type'] = ticket_summary.apply(get_primary_complaint, axis=1)

### Merge Datasets into Master Dataset

In [6]:
# Merge usage summary with customers
df_master = df_customers.merge(usage_summary, on='customer_id', how='left')

# Merge ticket summary
df_master = df_master.merge(ticket_summary, on='customer_id', how='left')

# Fill nulls for customers with no usage/tickets
# Fill usage-related nulls
usage_cols = ['active_days', 'total_logins', 'avg_logins_per_day', 'std_logins',
              'total_feature_events', 'avg_feature_events', 'total_session_minutes', 
              'avg_session_minutes', 'corrupted_records_count',
              'active_days_clean', 'total_logins_clean', 'avg_logins_per_day_clean',
              'total_session_minutes_clean']
for col in usage_cols:
    if col in df_master.columns:
        df_master[col] = df_master[col].fillna(0)

df_master['never_logged_in'] = df_master['never_logged_in'].fillna(True)
df_master['low_engagement'] = df_master['low_engagement'].fillna(True)
df_master['potential_outlier'] = df_master['potential_outlier'].fillna(False)

# Fill ticket-related nulls
ticket_cols = ['total_tickets', 'dashboard_issue_count', 'product_bug_count', 
               'onboarding_issue_count', 'avg_sentiment', 'avg_first_response_hours',
               'avg_resolution_hours', 'resolution_rate']
for col in ticket_cols:
    if col in df_master.columns:
        df_master[col] = df_master[col].fillna(0)

df_master['primary_complaint_type'] = df_master['primary_complaint_type'].fillna('No_Tickets')
df_master['high_ticket_volume'] = df_master['high_ticket_volume'].fillna(False)



/var/folders/vt/xrsm2sl9281d5zgcwfk59qbm0000gn/T/ipykernel_2690/4112780336.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_master['high_ticket_volume'] = df_master['high_ticket_volume'].fillna(False)


### Save Cleaned Data

In [7]:
df_master.to_csv('../data/rigorous_cleaned_master_data.csv', index=False)